## Q1. Does estimator has finite variance?
$f(x)=\frac{1}{\pi(1+x^2)}$<br>
$g(x)=\frac{exp(-x^2/2)}{sqrt(2\pi)}$<br>
$sup_x\frac{f(x)}{g(x)} = \infty$<br>
So we can't say surely about that as it is sufficient condition not a necessary one.<br>
So, let's try to simulate that one.


In [80]:
using Random
using QuadGK
using Distributions

In [86]:
f(x) = 1/(pi*(1+(x)^2))
g(x) = exp(-(x^2)/2)/sqrt(2*pi)
A = []
iterations = 10000
Random.seed!(1)
for j=0:iterations
    Z = rand(Normal())
    push!(A, (Z*f(Z))/g(Z))
end
var(A)

343.58217704429177

The variance value of estimator is finite.But it has a very high variance.If i don't set the seed variation is much more in value.

## Q2.

(a) From what we have seen in lectures, the variance of the weighted estimator is $\sigma_g^2(E_g[w(Z)^2])$ after some approximations. As w(Z)< M, we can say the expection will have lower bound but from this we can't make conclusions about it being finite.So it may be the case that it is infinity if expectation doesn't diverge.

(b) Yes, there is ofcourse a benefit as we reach to estimate fast as compared with Accept-reject where to choose even a single value we might have to perform several repetitions.

### Q3 IMPORTANCE SAMPLING ESTIMATOR

In [95]:
function sampler(v)
    Random.seed!(1)
    n = 50
    iterations = 5000
    y = rand(TDist(v),n)
    p(x) = exp(-(x^2)/2)*prod((1 .+((y.-x).^2)./v).^(-(1+v)/2))
    integral, err = quadgk(p,-Inf,Inf);
    constant = 1/integral
    pie(x) = constant*p(x)
    g(x) = exp(-(x^2)/2)/sqrt(2*pi)
    A = []
    for i=0:iterations
        Z = rand(Normal())
        push!(A,(Z*pie(Z)/g(Z)))
    end
    println("First moment  for Simple Sampler for v = ", v , " is ", mean(A))
    B = []
    W = []
    for i=0:iterations
        Z = rand(Normal())
        push!(B,(Z*p(Z)/g(Z)))
        push!(W,p(Z)/g(Z))
    end
    println("First Moment for Weighted Sampler for v = ", v , " is ", mean(B)/mean(W))
    println("Variance  for Weighted Sampler for v = ", v , " is ",var(A)*(1+var(W)/((mean(B)/mean(W))^2)))
end
sampler(5)

First moment  for Simple Sampler for v = 5 is 0.3258237240941205
First Moment for Weighted Sampler for v = 5 is 0.31633315800844575
Variance  for Weighted Sampler for v = 5 is 0.4529788658503306


In [96]:
sampler(1)
sampler(2)

First moment  for Simple Sampler for v = 1 is 0.1644603969092262
First Moment for Weighted Sampler for v = 1 is 0.16405313522093382
Variance  for Weighted Sampler for v = 1 is 0.14749586313632457
First moment  for Simple Sampler for v = 2 is 0.03699876516501468
First Moment for Weighted Sampler for v = 2 is 0.03974466782226018
Variance  for Weighted Sampler for v = 2 is 0.06811952857446894


For v = 1,2 we get a little less variance especially for v=2.

### Q.4 Posterior Distribution of $\lambda$

$\pi(\lambda) \propto \lambda^{\alpha-1} e^{-\beta\lambda}$<br>
$f(Y_1,Y_2,....,Y_n|\lambda) = \prod_{i=0}^{n}Poisson(\lambda) = \prod_{i=0}^{n}(\frac{\lambda^{y_i} e^{-\lambda}}{y_i!}) \propto \lambda^{\sum y_i} e^{-n\lambda}$<br>
$f(\lambda|Y_1,Y_2,....,Y_n) \propto {\pi(\lambda)f(Y_1,Y_2,....,Y_n|\lambda)}$<br>
$f(\lambda|Y_1,Y_2,....,Y_n) \propto \lambda^{\alpha+\sum y_i-1} e^{-(\beta+n)\lambda}$<br>
Thus, the posterior function will itself be a Gamma($\alpha+\sum y_i,\beta+n$) Distribution